# 将训练脚本作为命令作业运行

可使用适用于 Azure 机器学习的 Python SDK 将脚本作为命令作业提交。 通过使用作业，可在训练机器学习模型时轻松跟踪输入参数和输出。

## 准备工作

需要最新版本的“azureml-ai-ml”包才能运行此笔记本中的代码。 运行下面的单元以验证是否已安装它。

> **注意**：
> 如果未安装“azure-ai-ml”包，请运行 `pip install azure-ai-ml` 以进行安装。

In [ ]:
## 连接到工作区

安装必需的 SDK 包后，就可以连接到工作区了。

若要连接到工作区，我们需要标识符参数 - 订阅 ID、资源组名称和工作区名称。 已为你填写资源组名称和工作区名称。 只需订阅 ID 即可完成命令。

若要查找所需的参数，请单击工作室右上角的订阅和工作区名称。 右侧将打开一个窗格。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 复制订阅 ID，并将“YOUR-SUBSCRIPTION-ID”替换为复制的值。 </p>

## 使用 MLflow 进行自定义跟踪

将脚本作为作业运行时，可以在训练脚本中使用 MLflow 来跟踪模型。 MLflow 允许跟踪需要随作业输出存储的自定义参数、指标或项目。

运行以下单元格，在 src 文件夹中创建 train-model-mlflow.py 脚本 。 该脚本通过使用同一文件夹中的 diabetes.csv 文件（作为参数传递）来训练分类模型。 

查看以下代码，将看到脚本将导入 `mlflow` 并记录：

- 正则化率（作为参数）。 
- 准确度和 AUC（作为指标）。
- 绘制的 ROC 曲线（作为项目）。

In [ ]:
现在可以将脚本作为命令作业提交。

运行下面的单元格来训练模型。 

In [ ]:
在工作室中导航到“diabetes-train-mlflow”作业，浏览所运行的命令作业的概述：

- 在“概述”选项卡的“参数”下找到记录的参数 。
- 在“指标”选项卡中查找记录的指标。
- 在针对图像的“图像”选项卡和针对所有文件的“输出 + 日志”选项卡中找到记录的项目 。

## 使用 MLflow 自动记录日志

MLflow 还可以自动记录参数、指标和项目，而不使用自定义日志记录。 要使用 MLflow 自动记录日志，只需要一行代码。

运行以下单元格，在 src 文件夹中创建 train-model-autolog.py 脚本 。 该脚本通过使用同一文件夹中的 diabetes.csv 文件（作为参数传递）来训练分类模型。 

查看以下代码，可以看到脚本将导入 `mlflow` 并通过这行代码启用自动日志记录： 

`mlflow.autolog()`

In [ ]:
现在可以将脚本作为命令作业提交。

运行下面的单元格来训练模型。 

In [ ]:
在工作室中导航到“diabetes-train-autolog”作业，浏览所运行的命令作业的概述：

- 在“概述”选项卡的“参数”下找到记录的参数 。
- 在“指标”选项卡中查找记录的指标。
- 在针对图像的“图像”选项卡和针对所有文件（包括模型文件）的“输出 + 日志”选项卡中找到记录的项目 。

## 使用 MLflow 查看和搜索试验

Azure 机器学习工作室是一种易于使用的 UI，用于查看和比较作业运行。 或者，可以使用 MLflow 查看试验作业。 

若要列出工作区中的作业，请使用以下命令列出工作区中的试验：


In [ ]:
若要检索特定试验，可按名称获取试验：

使用试验名称，可以检索该试验的所有作业：

若要更轻松地比较作业运行和输出，可以配置搜索以对结果进行排序。 例如，以下单元格按 `start_time` 对结果进行排序，且仅显示最多 `2` 个结果： 

In [ ]:
甚至可以创建查询来筛选运行。 筛选器查询字符串是使用 SQL `WHERE` 子句的简化版本编写的。 

若要进行筛选，可以使用两类比较运算符：

- 数值比较运算符（指标）：=、!=、>、>=、< 和 <=。
- 字符串比较运算符（参数、标记和属性）：= 和 !=。

详细了解[如何通过 MLflow 跟踪试验](https://learn.microsoft.com/azure/machine-learning/how-to-track-experiments-mlflow)。

Now, you can submit the script as a command job.

Run the cell below to train the model. 

In [ ]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-autolog.py --training_data diabetes.csv",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-autolog",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

In the Studio, navigate to the **diabetes-train-autolog** job to explore the overview of the command job you ran:

- Find the logged parameters in the **Overview** tab, under **Params**.
- Find the logged metrics in the **Metrics** tab.
- Find the logged artifacts in the **Images** tab (specifically for images), and in the **Outputs + logs** tab (all files, including the model files).

## Use MLflow to view and search for experiments

The Azure Machine Learning Studio is an easy-to-use UI to view and compare job runs. Alternatively, you can use MLflow to view experiment jobs. 

To list the jobs in the workspace, use the following command to list the experiments in the workspace:


In [ ]:
import mlflow
experiments = mlflow.list_experiments()
for exp in experiments:
    print(exp.name)

To retrieve a specific experiment, you can get it by its name:

In [ ]:
experiment_name = "diabetes-training"
exp = mlflow.get_experiment_by_name(experiment_name)
print(exp)

Using an experiment name, you can retrieve all jobs of that experiment:

In [ ]:
mlflow.search_runs(exp.experiment_id)

To more easily compare job runs and outputs, you can configure the search to order the results. For example, the following cell orders the results by `start_time`, and only shows a maximum of `2` results: 

In [ ]:
mlflow.search_runs(exp.experiment_id, order_by=["start_time DESC"], max_results=2)

You can even create a query to filter the runs. Filter query strings are written with a simplified version of the SQL `WHERE` clause. 

To filter, you can use two classes of comparators:

- Numeric comparators (metrics): =, !=, >, >=, <, and <=.
- String comparators (params, tags, and attributes): = and !=.

Learn more about [how to track experiments with MLflow](https://learn.microsoft.com/azure/machine-learning/how-to-track-experiments-mlflow).

In [ ]:
query = "metrics.AUC > 0.8 and tags.model_type = 'LogisticRegression'"
mlflow.search_runs(exp.experiment_id, filter_string=query)